In [ ]:
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open('Confeitaria').sheet1

In [ ]:
# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
df=pd.DataFrame(rows[1:],columns=rows[0])

In [ ]:
intencoes = ['ver_cardapio', 'fazer_pedido', 'adicionar_ingrediente', 'tipo_decoracao', 'forma_pagamento']
xtrain_global = []
ytrain_global = []
for intencao in intencoes:
    lintencao = df[df['Conjunto']=='Treino'][intencao].values.tolist()
    xtrain_global += lintencao
    ytrain_global += [intencao]*len(lintencao)

In [ ]:
xtrain_global,ytrain_global

In [ ]:
xtest = []
ytest = []
for intencao in intencoes:
    lintencao = df[df['Conjunto']=='Teste'][intencao].values.tolist()
    xtest += lintencao
    ytest += [intencao]*len(lintencao)

In [ ]:
len(xtest),len(xtrain_global)

In [ ]:
xtest,ytest

In [ ]:
import sklearn.model_selection as model_selection

In [ ]:
xtrain,xval,ytrain,yval = model_selection.train_test_split(xtrain_global,ytrain_global,test_size=0.25,stratify=ytrain_global)

In [ ]:
np.unique(ytrain,return_counts=True)

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
converter = SentenceTransformer('multi-qa-distilbert-cos-v1')

In [ ]:
xtrain[:3]

In [ ]:
xval[:3]

In [ ]:
xtest[:5]

In [ ]:
xtrain_emb = converter.encode(xtrain)
xval_emb = converter.encode(xval)
xtest_emb = converter.encode(xtest)

In [ ]:
xtrain[1],xtrain_emb[1][:10]

In [ ]:
import sklearn.neighbors as neighbors

In [ ]:
model = neighbors.KNeighborsClassifier(n_neighbors=5)

In [ ]:
model.fit(xtrain_emb,ytrain)

In [ ]:
import sklearn.metrics as metrics

In [ ]:
for i in range(1,15,2):
    model = neighbors.KNeighborsClassifier(n_neighbors=i,weights="distance")
    model.fit(xtrain_emb,ytrain)
    pred = model.predict(xval_emb)
    print(f"k={i}")
    print(metrics.classification_report(yval,pred))

# Avaliação do Modelo

In [ ]:
import sklearn.metrics as metrics

In [ ]:
model = neighbors.KNeighborsClassifier(n_neighbors=9,weights="distance")
model.fit(xtrain_emb,ytrain)
pred = model.predict(xtest_emb)
print(f"k={i}")
print(metrics.classification_report(ytest,pred))

# Teste

In [ ]:
model.predict([converter.encode('Morango')])

#Modelo final

In [ ]:
model.fit(converter.encode(xtrain_global+xtest),ytrain_global+ytest)

In [ ]:
from joblib import dump, load

In [ ]:
dump(model, 'Confeitaria.joblib') 

In [ ]:
modelo_final = load('Confeitaria.joblib') 

In [ ]:
modelo_final.predict([converter.encode('Me ve o menu')])

# Construção de um chatbot

In [ ]:
def convert_num(n):
    valores = {'um':1,
               'uma':1,
               'dois':2,
               'duas':2,
               'tres':3,
               'quatro':4,
               'cinco':5,
               'seis':6,
               'sete':7,
               'oito':8,
               'nove':9,
               'dez':10}
    ret = 0
    if n.isnumeric():
        ret = int(n)
    else:
        if n in valores.keys():
            ret = valores[n]

    return ret

In [ ]:
!pip install unidecode

In [ ]:
from nltk.tokenize import TweetTokenizer
from unidecode import unidecode
tknzr = TweetTokenizer()

In [ ]:
lista_entidades = [
'item:chocolate,branco,camafeu,limao',
'adicional:morango,nozes,ameixa',
'decoracao:chantilly,brigadeiro',
'num:1,2,3,4,5,6,7,8,9,10',
'num:um,dois,tres,quatro,cinco,seis,sete,oito,nove,dez,uma,duas'
]
entidades = dict()
def load_entidades(lista_entidades):
            for line in lista_entidades:
                entidade,valores = line.split(':')
                str_valores = valores[:]
                valores = str_valores.split(',')
                for valor in valores:
                    if valor not in entidades.keys():
                        entidades[valor] = entidade
load_entidades(lista_entidades)
def find_entidades(texto):
    ret = dict()
    for token in tknzr.tokenize(texto):
        token = token.lower()
        token = unidecode(token)
        if token in entidades.keys():
            ent = entidades[token]
            if ent not in ret.keys():
                ret[ent] = [token]
            else:
                ret[ent] += [token]
    return ret

In [ ]:
def convert_num(n):
    valores = {'': 1,
               'um':1,
               'uma':1,
               'dois':2,
               'duas':2,
               'tres':3,
               'quatro':4,
               'cinco':5,
               'seis':6,
               'sete':7,
               'oito':8,
               'nove':9,
               'dez':10}
    ret = 0
    if n.isnumeric():
        ret = int(n)
    else:
        if n in valores.keys():
            ret = valores[n]

    return ret

In [ ]:
print(find_entidades('Decoração de brigadeiro'))

In [ ]:
cardapio_item = {
    'chocolate':20.0,
    'branco':20.0,
    'camafeu':20.0,
    'limao':20.0    
}
cardapio_adicional={
    'morango':10.0,
    'nozes':10.0,
    'ameixa':10.0,
}
cardapio_decoracao={
    'chantilly':7.0,
    'brigadeiro':7.0
}

In [ ]:
def str_menu(h):
    rstr = ''
    for item in h:
        rstr += f"{item:<10}  {h[item]:>5}\n"
    return rstr


In [ ]:
print(str_menu(cardapio_item))


In [ ]:
print(str_menu(cardapio_adicional))

In [ ]:
print(str_menu(cardapio_decoracao))

In [ ]:
!pip install pytelegrambotapi --upgrade

In [ ]:
import telebot

In [ ]:
from re import M
bot = telebot.TeleBot("5056778674:AAFwZIZHn7YXQEdYE-GXZ3miA_RMqc0rCus")
pedidos = []

@bot.message_handler(commands=["start"])
def inicia(mensagem_usuario):
    bot.send_message(mensagem_usuario.chat.id)

@bot.message_handler()
def responder(mensagem_usuario):
    cliente = mensagem_usuario.text
    
    msg = ''
    pred = modelo_final.predict([converter.encode(cliente)])[0]

    if pred == 'ver_cardapio':
      msg= 'Ola, seja bem vindo a confeitaria amor em pedaços\n\n****BOLOS****\n\n'
      msg+= str_menu(cardapio_item)
      msg+='\n****ADICIONAL****\n\n'
      msg+= str_menu(cardapio_adicional)
      msg+='\n****DECORAÇÕES****\n\n'
      msg+= str_menu(cardapio_decoracao)
      msg+='\n\nO que gostaria de pedir?'
    
    elif pred == 'fazer_pedido':
        ent = find_entidades(cliente)
        print(ent)
        joined = [[x,y] for x,y in zip(ent['num'], ent['item'])]
        pedidos.append(joined)
        msg += 'Ok, '
        for n,item in joined[:-1]:
            msg += ',%s %s '%(n,item)
        msg += ('%s bolo sabor %s saindo'%(joined[-1][0],joined[-1][1]))
        msg +='\n\nGostaria de adicionar alguma de nossas opções ao seu recheio?\nCaso não queira, qual cobertura você escolhe?'       

    elif pred == 'adicionar_ingrediente':
        ent = find_entidades(cliente)
        print(ent)
        joined = [[x,y] for x,y in zip('1', ent['adicional'])]
        pedidos.append(joined)
        msg += 'O adicional no recheio será de '
        for n,adicional in joined[:-1]:
            msg += ' ,%s'%(adicional)
        msg += ('%s '%(joined[-1][1]))        
        msg +='\n\nQual cobertura você escolhe?'  

    elif pred == 'tipo_decoracao':
        ent = find_entidades(cliente)
        print(ent)
        joined = [[x,y] for x,y in zip('1', ent['decoracao'])]
        pedidos.append(joined)
        msg += 'Seu bolo sera decorado com '
        for n,decoracao in joined[:-1]:
            msg += ' ,%s'%(decoracao)
        msg += ('%s '%(joined[-1][1]))        
        msg+='\n\nComo deseja pagar?'

    elif pred == 'forma_pagamento':
        msg += 'Ok, foi pedido:\n'
        total = 0
        for joined in pedidos:
            for n,item in joined:
                nvalor = convert_num(n)
                if item in cardapio_item.keys(): 
                    vitem = cardapio_item[item]*nvalor
                if item in cardapio_adicional.keys():
                    vitem = cardapio_adicional[item]
                if item in cardapio_decoracao.keys():
                    vitem = cardapio_decoracao[item]
                total += vitem
                msg += '%s %s %04.2f\n'%(n,item,vitem)
        msg += ('Total foi R$ %4.2f\n'%total)

    elif pred == 'bye':
        msg = 'Bye, volte sempre\n'
        sair = True

    else:
        msg = 'Não, entendi, veja nossas opções.\n\n****BOLOS****\n\n'
        msg+= str_menu(cardapio_item)
        msg+='\n****ADICIONAL****\n\n'
        msg+= str_menu(cardapio_adicional)
        msg+='\n****DECORAÇÕES****\n\n'
        msg+= str_menu(cardapio_decoracao)
        msg+='\n\nO que gostaria de pedir?'
    
    bot.send_message(mensagem_usuario.chat.id, msg)

bot.polling()